# Predict revenue per customer for a Google Merchandise store.
### **What is Revenue:**

It is total revenue in a given time period by the number of users in that time period.

**Revenue: ** **Total Revenue/ Number of Users.**


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

### Read train.csv file 


In [ ]:

df = pd.read_csv('../input/train.csv') #parse_dates=['date'])

In [ ]:
df.head()

## Data Fields: Column Information

fullVisitorId- A unique identifier for each user of the Google Merchandise Store. <br>
channelGrouping - The channel via which the user came to the Store. <br>
date - The date on which the user visited the Store.<br>
device - The specifications for the device used to access the Store. <br>
geoNetwork - This section contains information about the geography of the user. <br>
sessionId - A unique identifier for this visit to the store.<br>
socialEngagementType - Engagement type, either "Socially Engaged" or "Not Socially Engaged". <br>
totals - This section contains aggregate values across the session.<br>
trafficSource - This section contains information about the Traffic Source from which the session originated. <br>
visitId - An identifier for this session. <br>
visitNumber - The session number for this user. If this is the first session, then this is set to 1.<br>
visitStartTime - The timestamp (expressed as POSIX time).<br>


### Processing JSON data.
From data description we can see that following columns 'device', 'geoNetwork', 'totals' and 'trafficSource' contains data in JSON format. 
JSON library can parse JSON data into string or files. The library parse data into python dictionary or strings. 


In [ ]:
# Create new dataframes from JSON fields

device_df = df.device.apply(json.loads).values.tolist()
device_df = pd.DataFrame.from_records(device_df)

geoNetwork_df = df.geoNetwork.apply(json.loads).values.tolist()
geoNetwork_df = pd.DataFrame.from_records(geoNetwork_df)

totals_df = df.totals.apply(json.loads).values.tolist()
totals_df = pd.DataFrame.from_records(totals_df)

trafficSource_df = df.trafficSource.apply(json.loads).values.tolist()
trafficSource_df = pd.DataFrame.from_records(trafficSource_df)


Drop 'device', 'geoNetwork', 'totals', 'trafficSource' columns from original datset. 
Merge main and parsed dataset. 

In [ ]:
df.drop(['device', 'geoNetwork', 'totals', 'trafficSource'], axis=1, inplace=True)
df = pd.concat([df, geoNetwork_df, device_df, totals_df, trafficSource_df], axis=1)

In [ ]:
df.head()

In [ ]:
df.info()

Check for null values acrocss each column and sum the total number of null values. 

In [ ]:
df.isnull().sum(axis = 0)
#print(df.dtypes)

In [ ]:
df.channelGrouping.unique()

In [ ]:
df.fullVisitorId.unique()

## **geoNetworK**:
It contains logs of Geographical details of users with 10 attributes.
<br>
One: City – Paris, Dubai, Seattle. <br>
Two: City ID – Not available.<br>
Three: Continent –Asia, Europe, America.<br>
Four: Country – USA, India, France.<br>
Five: Latitude – Not Available. <br>
Six: Longitude – Not Available. <br>
Seven: Metro – Chicago IL, Las Vegas NV. <br>
Eight: Network Domain – malnet.ru, lumenis.com <br>
Network Location: Not Available. <br>
Nine: Region – Sindh, Karnataka, Moscow. <br>
Ten: subcontinent: South Asia, North Africa. <br>    


In [ ]:
for col in geoNetwork_df:
    print (col,geoNetwork_df[col].unique())

## **Device:**
It contains logs of electronic device used by use: Attributes are: <br>
First: Browser – Chrome, Opera Mini.<br>
Second: BrowserSize – Not Available.<br>
Third: BrowserVersion : Not Available.<br>
Forth: DeviceCategory: Desktop, mobile, tablet.<br>
Fifth: FlashVersion: Not Available.<br>
Sixth: IsMobile – False, True.<br>
Seventh: Language – Not Available.<br>
Eighth: mobileDeviceBranding: Not Available.<br>
Ninth: MobileDeviceInfo – Not available.<br>
Tenth: MobileDeviceMarketingName: Not Available<br>
Eleventh : mobileDeviceModel : Not Available <br>
Twelfth: mobileInputSelector : Not Available <br>
Thirteen: operatingSystem – Windows, Linux. <br>
Fourteen: operatingSystemVersion – Not Available <br>
Fifteen: screenColors – Not Available. <br>
Sixteen: screenResolution – Not available. <br>


In [ ]:
for col in device_df:
    print (col,device_df[col].unique())

In [ ]:
for col in totals_df:
    print (col,totals_df[col].unique())

In [ ]:
for col in trafficSource_df:
    print (col,trafficSource_df[col].unique())

In [ ]:
trafficSource_df.head()

In [ ]:
trafficSource_df['adwordsClickInfo'].head()

In [ ]:
#adwordsClickInfo_df = trafficSource_df.adwordsClickInfo.apply(json.loads).values.tolist()
adwordsClickInfo_df = pd.DataFrame.from_records(trafficSource_df['adwordsClickInfo'])
adwordsClickInfo_df.head()

## Remove unnecessary columns or colums containing least infomation
We can see that some columns contains no usful  infomation, we are going to drop those columns from main dataset.


In [ ]:
df.head()

In [40]:
df.drop(['latitude','longitude','cityId','networkLocation', 'browserSize','browserVersion',
        'flashVersion','language','mobileDeviceBranding','mobileDeviceInfo','mobileDeviceMarketingName',
        'mobileDeviceModel','mobileInputSelector','operatingSystemVersion','screenColors','screenResolution',
        'adwordsClickInfo'],axis=1)

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,city,continent,country,metro,networkDomain,region,subContinent,browser,deviceCategory,isMobile,operatingSystem,bounces,hits,newVisits,pageviews,transactionRevenue,visits,adContent,campaign,campaignCode,isTrueDirect,keyword,medium,referralPath,source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Izmir,Asia,Turkey,(not set),ttnet.com.tr,Izmir,Western Asia,Chrome,desktop,False,Windows,1,1,1,1,NaN,1,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,not available in demo dataset,Oceania,Australia,not available in demo dataset,dodo.net.au,not available in demo dataset,Australasia,Firefox,desktop,False,Macintosh,1,1,1,1,NaN,1,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Madrid,Europe,Spain,(not set),unknown.unknown,Community of Madrid,Southern Europe,Chrome,desktop,False,Windows,1,1,1,1,NaN,1,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,not available in demo dataset,Asia,Indonesia,not available in demo dataset,unknown.unknown,not available in demo dataset,Southeast Asia,UC Browser,desktop,False,Linux,1,1,1,1,NaN,1,NaN,(not set),NaN,NaN,google + online,organic,NaN,google
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,not available in demo dataset,Europe,United Kingdom,not available in demo dataset,unknown.unknown,not available in demo dataset,Northern Europe,Chrome,mobile,True,Android,1,1,NaN,1,NaN,1,NaN,(not set),NaN,True,(not provided),organic,NaN,google
5,Organic Search,20160902,2938943183656635653,2938943183656635653_1472807194,Not Socially Engaged,1472807194,1,1472807194,not available in demo dataset,Europe,Italy,not available in demo dataset,fastwebnet.it,not available in demo dataset,Southern Europe,Chrome,desktop,False,Windows,1,1,1,1,NaN,1,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
6,Organic Search,20160902,1905672039242460897,1905672039242460897_1472817241,Not Socially Engaged,1472817241,1,1472817241,not available in demo dataset,Asia,Pakistan,not available in demo dataset,unknown.unknown,not available in demo dataset,Southern Asia,Chrome,desktop,False,Windows,1,1,1,1,NaN,1,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
7,Organic Search,20160902,537222803633850821,537222803633850821_1472812602,Not Socially Engaged,1472812602,1,1472812602,Brisbane,Oceania,Australia,(not set),bigpond.net.au,Queensland,Australasia,Chrome,desktop,False,Windows,1,1,1,1,NaN,1,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
8,Organic Search,20160902,4445454811831400414,4445454811831400414_1472805784,Not Socially Engaged,1472805784,1,1472805784,not available in demo dataset,Europe,Austria,not available in demo dataset,spar.at,not available in demo dataset,Western Europe,Internet Explorer,desktop,False,Windows,1,1,1,1,NaN,1,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
9,Organic Search,20160902,9499785259412240342,9499785259412240342_1472812272,Not Socially Engaged,1472812272,1,1472812272,not available in demo dataset,Europe,Netherlands,not available in demo dataset,chello.nl,not available in demo dataset,Western Europe,Firefox,desktop,False,Windows,1,1,1,1,NaN,1,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
